In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
print ("uname:", platform.uname())

print ("\nsystem   :", platform.system())
print ("node     :", platform.node())
print ("release  :", platform.release())
print ("version  :", platform.version())
print ("machine  :", platform.machine())
print ("processor:", platform.processor())

In [ ]:
import subprocess
import pprint

sp = subprocess.Popen(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

out_str = sp.communicate()
out_list = str(out_str[0]).split('\\n')

out_dict = {}

for item in out_list:
    print(item)

In [ ]:
import subprocess
import pprint

sp = subprocess.Popen(['nvidia-smi', '-q'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

out_str = sp.communicate()
out_list = str(out_str[0]).split('\\n')

out_dict = {}

for item in out_list:
    print(item)

In [ ]:
import platform, subprocess

def get_processor_info():
    if platform.system() == "Windows":
        return platform.processor()
    elif platform.system() == "Darwin":
        return subprocess.check_output(['/usr/sbin/sysctl', "-n", "machdep.cpu.brand_string"]).strip()
    elif platform.system() == "Linux":
        command = "cat /proc/cpuinfo"
        return subprocess.check_output(command, shell=True).strip()
    return ""

get_processor_info()

## MNIST

In [2]:
# Disable GPU
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'

In [1]:
# Trains a simple convnet on the MNIST dataset

from __future__ import print_function
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import time

batch_size = 32
num_classes = 10
epochs = 3

# input image dimensions
img_rows, img_cols = 28, 28

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
input_shape

(28, 28, 1)

In [6]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [7]:
t1=time.time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
t2=time.time()

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 120s 2ms/step - loss: 0.1844 - acc: 0.9440 - val_loss: 0.0497 - val_acc: 0.9832
Epoch 2/3
60000/60000 [==============================] - 124s 2ms/step - loss: 0.0772 - acc: 0.9770 - val_loss: 0.0363 - val_acc: 0.9875
Epoch 3/3
60000/60000 [==============================] - 121s 2ms/step - loss: 0.0593 - acc: 0.9822 - val_loss: 0.0354 - val_acc: 0.9878


In [8]:
print('Time to train',(t2-t1))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Time to train 365.9802896976471
Test loss: 0.03544133635050967
Test accuracy: 0.9878
